In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import udf
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.mllib.linalg import Vectors
#from pyspark.ml.feature import VectorAssemblerg
from IPython.display import display
from ipywidgets import interact
import sys
import numpy as np
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import os.path
%matplotlib inline

In [2]:
import json
import time
#import simplejson as json
from datetime import datetime


In [3]:
def getTime(start):
    sec = time.time() - start
    m, s = divmod(sec, 60)
    h, m = divmod(m, 60)
    return s,m,h, sec

In [23]:

# full
data_raw = sc.textFile('/mnt/66e695cd-1a0c-4e3b-9a50-55e01b788529/Tweet_Output')

# test
#data_raw = sc.textFile('/mnt/66e695cd-1a0c-4e3b-9a50-55e01b788529/Tweet_Output/2013-02-01.json.gz')


#data_raw = sc.textFile('/resources/data/2013-06') 

In [24]:
data = data_raw.map(lambda line: json.loads(line))

In [25]:
sample = data.first()
sample

{u'HashTag_Birthday': 1372229881.0,
 u'from_user': u'RyanMuthiora',
 u'hashtag': [u'relationshipdealbreaker'],
 u'lang': u'en',
 u'location': [u'loc_'],
 u'mention': [],
 u'term': [u'girls', u'who', u'ride', u'horses']}

In [26]:
def is_error(line):
    return "ERROR" in line
errors = data.filter(is_error)

In [27]:
def parserHashTag(r):
    r['hashtag']
    return r['hashtag']

In [28]:
from operator import add

now = time.time()

hashtagRDD = data.flatMap(parserHashTag).map(lambda x: (x,1))
count = hashtagRDD.reduceByKey(add)

#sorted
Topk = count.takeOrdered(10, key=lambda x: -x[1])   # x[1] means sort by value.

s,m,h,sec = getTime(now)
print('Spark count takes - %d:%02d:%02d which is %d seconds in total' % (h,m,s,sec))
## asending order takeOrdered(10, key=lambda x: x[1])
print('Top 10 hashtags are:', Topk)

Spark count takes - 0:03:41 which is 221 seconds in total
('Top 10 hashtags are:', [(u'gameinsight', 2806104), (u'vote5sos', 1446189), (u'androidgames', 1377591), (u'android', 1327543), (u'TeamFollowBack', 1213607), (u'MTVHottest', 1138814), (u'RT', 1128746), (u'ipadgames', 1098291), (u'MTVStars', 1061970), (u'RETWEET', 1014617)])


In [ ]:
## note, the next four lines could be combined to avoid multiple lambda function, just separate here for clarity 

In [29]:
HashCount = data.flatMap(lambda line:  line['hashtag']).distinct().count()
print ('{} unique hashtags'.format(HashCount))

21708629 unique hashtags


In [30]:
FromUser = data.map(lambda line:  line['from_user']).distinct().count()
print ('{} unique from user'.format(FromUser))

37373442 unique from user


In [31]:
Term = data.flatMap(lambda line:  line['term']).distinct().count()
print ('{} unique terms in Tweet'.format(Term))

59531431 unique terms in Tweet


In [20]:
Mention = data.flatMap(lambda line:  line['mention']).distinct().count()
print ('{} unique mentions'.format(Mention))

168326 unique mentions


In [22]:
num_valid_twt = data.count()
print('Total number of valid tweets are {}'.format(num_valid_twt))

Total number of valid tweets are 327642


In [16]:
def parserMin(r):
    pair = []
    for hashtag in r['hashtag']:
        pair.append((hashtag, r['HashTag_Birthday']))
        
    return pair

def getMin(a,b):
    return a if a < b else b
parserMin(sample)

[(u'StandUpComedy', 1359662285.0),
 (u'Humor', 1359662285.0),
 (u'Magia', 1359662285.0)]

In [17]:
Hashtag_birthday = data.flatMap(parserMin).reduceByKey(getMin) # reduce tells you how to compare values in two lines.

In [18]:
Hashtag_birthday.take(10)

[(u'foramarien', 1359701262.0),
 (u'Aguaaas', 1360672153.0),
 (u'FortalezaEstouContigo', 1360023237.0),
 (u'whelanwhelan', 1360200220.0),
 (u'realsuper', 1359965201.0),
 (u'ClausuroIndecPorq', 1360264871.0),
 (u'coollime', 1361870803.0),
 (u'DefensaContraQuerellaLeonel', 1360384766.0),
 (u'elpaisqueq', 1361868450.0),
 (u'ElAsesino', 1360006234.0)]

In [32]:
sample2 = data.first()

In [28]:
def parserAll(r):
    try:
        x=Row(Birthday=float(r['HashTag_Birthday']),\
          UserName= str(r['from_user']), \
          Hashtag= r['hashtag'],\
          Language=r['lang'],\
          Location=r['location'], \
          Mention=r['mention'],\
          Term=r['term'])  
    except:
        x=None  
    return x

rowRDD = data.map(lambda r: parserAll(r))

In [29]:
rowRDD.first()

Row(Birthday=1370768309.0, Hashtag=[u'iloveyou', u'missyou'], Language=u'en', Location=[u'loc_Indonesia'], Mention=[u'aku_jujur'], Term=[u'RT', u'@aku_jujur:', u'Jika', u'bisa,', u'berikan', u'aku', u'mimpi', u'tentangnya,', u'dan', u'jika', u'berhasil', u'tolong', u'jangan', u'bangunkan', u'aku', u'#iloveyou', u'#missyou'], UserName='Anasya_Bella')

In [34]:
type(sample2)

dict

In [116]:
Feature_df = sqlContext.createDataFrame(rowRDD)

TypeError: data is already a DataFrame

In [31]:
!pip install flatten_json

  Running setup.py bdist_wheel for flatten-json ... - \ | done
  Stored in directory: /home/notebook/.cache/pip/wheels/e4/ab/04/38966a4fb96e02c81dcb926a43a787f437c6111f1c69b0353a
Successfully built flatten-json
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [38]:
import flatten_json as fj
fj.flatten_json(sample2)

{u'HashTag_Birthday': 1370768309.0,
 u'from_user': u'Anasya_Bella',
 'hashtag_0': u'iloveyou',
 'hashtag_1': u'missyou',
 u'lang': u'en',
 'location_0': u'loc_Indonesia',
 'mention_0': u'aku_jujur',
 'term_0': u'RT',
 'term_1': u'@aku_jujur:',
 'term_10': u'berhasil',
 'term_11': u'tolong',
 'term_12': u'jangan',
 'term_13': u'bangunkan',
 'term_14': u'aku',
 'term_15': u'#iloveyou',
 'term_16': u'#missyou',
 'term_2': u'Jika',
 'term_3': u'bisa,',
 'term_4': u'berikan',
 'term_5': u'aku',
 'term_6': u'mimpi',
 'term_7': u'tentangnya,',
 'term_8': u'dan',
 'term_9': u'jika'}

In [129]:
Feature_df = df2
Feature_df.head(1)

[Row(HashTag_Birthday=1370768309.0, from_user=u'Anasya_Bella', hashtag=[u'iloveyou', u'missyou'], lang=u'en', location=[u'loc_Indonesia'], mention=[u'aku_jujur'], term=[u'RT', u'@aku_jujur:', u'Jika', u'bisa,', u'berikan', u'aku', u'mimpi', u'tentangnya,', u'dan', u'jika', u'berhasil', u'tolong', u'jangan', u'bangunkan', u'aku', u'#iloveyou', u'#missyou'])]

In [130]:
flat1 = Feature_df.withColumn('Hashtag', explode('Hashtag'))

In [131]:
flat1.registerTempTable("FeatureDF")

In [132]:
flat1.first()

Row(HashTag_Birthday=1370768309.0, from_user=u'Anasya_Bella', Hashtag=u'iloveyou', lang=u'en', location=[u'loc_Indonesia'], mention=[u'aku_jujur'], term=[u'RT', u'@aku_jujur:', u'Jika', u'bisa,', u'berikan', u'aku', u'mimpi', u'tentangnya,', u'dan', u'jika', u'berhasil', u'tolong', u'jangan', u'bangunkan', u'aku', u'#iloveyou', u'#missyou'])

In [138]:
from pyspark.sql.functions import countDistinct, approxCountDistinct
flat1.agg(countDistinct(flat1.Hashtag).alias('HashTag_Count')).collect()

[Row(HashTag_Count=2217643)]

In [137]:
Feature_df.agg(countDistinct(Feature_df.from_user).alias('From_Count')).collect()

[Row(From_Count=5917584)]

In [158]:
flat2 = Feature_df.withColumn('mention', explode('mention'))  ## bug...? mention return different countdistinct from_user

In [159]:
flat2.agg(countDistinct(flat2.mention).alias('From_Count')).collect()

[Row(From_Count=2503011)]

In [53]:
eDF = sqlContext.createDataFrame([Row(a=1, intlist=[1,2,3])])

In [60]:
from pyspark.sql.functions import explode

In [62]:
flat = eDF.withColumn('intlist', explode('intlist'))

In [63]:
flat.show()

+---+-------+
|  a|intlist|
+---+-------+
|  1|      1|
|  1|      2|
|  1|      3|
+---+-------+



In [9]:
Feature_df.registerTempTable("FeatureDF")

In [ ]:
groupedDelay = sqlContext.sql("SELECT Origin, count(*) conFlight,avg(DepDelay) delay \
                                FROM FeatureDF \
                                GROUP BY Origin")

In [53]:
data = sc.parallelize( [(0, 2.), (0, 4.), (1, 0.), (1, 10.), (1, 20.)] )


In [55]:
data.map(lambda value: (value, 1)).collect()

[((0, 2.0), 1), ((0, 4.0), 1), ((1, 0.0), 1), ((1, 10.0), 1), ((1, 20.0), 1)]

In [ ]:

sumCount = data.combineByKey(lambda value: (value, 1),
                             lambda x, value: (x[0] + value, x[1] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1]))

averageByKey = sumCount.map(lambda (label, (value_sum, count)): (label, value_sum / count))

print averageByKey.collectAsMap()

In [ ]:
       x=Row(Birthday=float(r['HashTag_Birthday']),\
          UserName= str(r['from_user']), \
          Hashtag= r['hashtag'],\
          Language=r['lang'],\
          Location=r['location'], \
          Mention=r['mention'],\
          Term=r['term']) 

In [41]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("hashtag", StringType(), True),
    StructField("HashTag_Birthday", FloatType(), True),
    StructField("from_user", StringType(), True),
    StructField("lang", StringType(), True),
    StructField("location", StringType(), True),
    StructField("mention", StringType(), True),
    StructField("term", StringType(), True)])


In [127]:
df2 = sqlContext.read.json('/resources/data/ProcessesTweeter')
df2.printSchema()

root
 |-- HashTag_Birthday: double (nullable = true)
 |-- from_user: string (nullable = true)
 |-- hashtag: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lang: string (nullable = true)
 |-- location: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- mention: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- term: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [128]:
df2.first()

Row(HashTag_Birthday=1370768309.0, from_user=u'Anasya_Bella', hashtag=[u'iloveyou', u'missyou'], lang=u'en', location=[u'loc_Indonesia'], mention=[u'aku_jujur'], term=[u'RT', u'@aku_jujur:', u'Jika', u'bisa,', u'berikan', u'aku', u'mimpi', u'tentangnya,', u'dan', u'jika', u'berhasil', u'tolong', u'jangan', u'bangunkan', u'aku', u'#iloveyou', u'#missyou'])

In [44]:
record = {
    'first_name': 'nadbor',
    'last_name': 'drozd',
    'occupation': 'data scientist',
    'children': [
        {
            'name': 'Lucja',
            'age': 3,
            'likes cold showers': True
        }
    ]
}

In [45]:
from pyspark.sql.types import StringType, StructField, StructType, BooleanType, ArrayType, IntegerType
schema = StructType([
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("occupation", StringType(), True),
        StructField("children", ArrayType(
            StructType([
                StructField("name", StringType(), True),
                StructField("age", IntegerType(), True),
                StructField("likes cold schowers", BooleanType(), True)
            ])
        ), True)
    ])